# 3. 데이터 전처리 이해와 실무

### 3.2 데이터 전제 : 이상치 데이터 처리

### 이상치 다루기 (강의 교안)

1. 이상치 데이터 확인
- Z-score, IQR(interquartile Range)
2. 이상치 처리
- 삭제, 대체

### 실습 내용 요약
- 예제 데이터 내 이상치 확인 및 삭제/대체방안 실습

### 실습 데이터
- 이상치 강좌 실습을 위한 생성데이터 : 기사별 클릭 수 데이터

### 실습시 활용 패키지
- pandas
- numpy
- scipy|

In [1]:
import pandas as pd
import numpy as np

In [4]:
# 데이터 로딩 및 개요 확인
# 실습데이터 제공받는다면 그 데이터로 해보기
# click_data = pd.read_csv("./data/click_sample_data.csv", encoding = 'cp949')
# click_data.head

# 실습데이터 임의로 만들기
np.random.seed(1)
cat_list = ['사설', '사회', '공학', '증권', '부동산', '정치', '스포츠']
cat = np.random.randint(0,7, size=10000)

np.random.seed(2)
jour_list = ['A일보', 'B일보', 'C일보', 'D일보', 'E일보']
jour = np.random.randint(0,5, size=10000)

np.random.seed(3)
num_click = np.random.randint(9, 3000, size=10000)

make_data = []
for i in range(10000):
    make_data.append([cat_list[cat[i]], jour_list[jour[i]], num_click[i]])
    
# 이상치 만들기
make_data[10][2] = 9
make_data[1001][2] = 68
make_data[1011][2] = 113
make_data[2003][2] = 79
make_data[9000][2] = 433992
make_data[900][2] = 433992
make_data[9003][2] = 433992
make_data[9004][2] = 433992
make_data[9005][2] = 233992
make_data[9006][2] = 333992
make_data[9007][2] = 333992

#만든 데이터를 데이터프레임으로
click_data = pd.DataFrame(np.array(make_data))
click_data.columns = ['category', 'Journal', 'num_click']
click_data['num_click'] = click_data['num_click'].astype(float)
click_data



,category,Journal,num_click
0,정치,A일보,1907.0
1,증권,A일보,1697.0
2,부동산,D일보,1676.0
3,사설,C일보,977.0
4,사회,D일보,2313.0
...,...,...,...
9995,사회,B일보,2184.0
9996,정치,D일보,1080.0
9997,사회,A일보,2583.0
9998,사회,B일보,2151.0


In [5]:
# 데이터 개오 파악
click_data.info()

# 데이터 카피
click_copy = click_data.copy()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   category   10000 non-null  object 
 1   Journal    10000 non-null  object 
 2   num_click  10000 non-null  float64
dtypes: float64(1), object(2)
memory usage: 234.5+ KB


In [6]:
# describe 함수 활용 긴반의 수치형 변수인 num_click 컬럼 요약 통계확인
click_copy['num_click'].describe()

count     10000.000000
mean       1764.480600
std       10150.132161
min           9.000000
25%         738.750000
50%        1522.500000
75%        2251.000000
max      433992.000000
Name: num_click, dtype: float64

- 최대 클릭수는 433992회 최소 9회 클릭되었음

- 75% 수준의 클릭수는 1000회 정도의 결과를 보임에 따라, 일부 이상치 데이터가 존재하는 것으로 보여짐

## 이상치 처리하기

### 이상치 확인 방안 및 삭제하기

1. Z-Score
2. IQR

- Z-score 수식
    - z = (x - meen)(std.dev)
    - z = (해당 관측치 - 관측치 변수 평균)/(관측치 변수의 표준편차)

In [7]:
# Z-score 컬럼 생성
click_copy['z-score'] = (click_copy['num_click'] - np.mean(click_copy['num_click'])) / np.std(click_copy['num_click'])
click_copy.head()

,category,Journal,num_click,z-score
0,정치,A일보,1907.0,0.014042
1,증권,A일보,1697.0,-0.006649
2,부동산,D일보,1676.0,-0.008718
3,사설,C일보,977.0,-0.077587
4,사회,D일보,2313.0,0.054043


In [8]:
# 원본 데이터 내 z-score 확인
click_copy.describe()

,num_click,z-score
count,10000.000000,1.000000e+04
mean,1764.480600,-4.938203e-17
std,10150.132161,1.000050e+00
min,9.000000,-1.729601e-01
25%,738.750000,-1.010609e-01
50%,1522.500000,-2.384133e-02
75%,2251.000000,4.793472e-02
max,433992.000000,4.258557e+01


In [9]:
# 이상치 처리하기 [삭제]
# z-score 기반 이상치 제거 후 데이터 차원확인
click_copy = click_copy[(click_copy['z-score'] < 3) & (click_copy['z-score'] > -3)]

# 데이터 개요 확인
click_copy.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9993 entries, 0 to 9999
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   category   9993 non-null   object 
 1   Journal    9993 non-null   object 
 2   num_click  9993 non-null   float64
 3   z-score    9993 non-null   float64
dtypes: float64(2), object(2)
memory usage: 390.4+ KB


In [10]:
# 이상치 제거후 데이터 요약 통계 확인
click_copy.describe()

,num_click,z-score
count,9993.000000,9993.000000
mean,1501.737416,-0.025887
std,866.173486,0.085340
min,9.000000,-0.172960
25%,738.000000,-0.101135
50%,1521.000000,-0.023989
75%,2249.000000,0.047738
max,2999.000000,0.121632


In [13]:
# Scipy 내 함수 제공

from scipy.stats import zscore

click_copy = click_data.copy()

In [18]:
click_copy['z-score'] = zscore(click_copy['num_click'])
click_copy = click_copy[(click_copy['z-score'] < 3) & (click_copy['z-score'] > -3)]
print(click_copy.shape)

(9993, 4)


In [19]:
# 기존 결과와 비교
click_copy.describe()

,num_click,z-score
count,9993.000000,9.993000e+03
mean,1501.737416,-5.433905e-17
std,866.173486,1.000050e+00
min,9.000000,-1.723456e+00
25%,738.000000,-8.817814e-01
50%,1521.000000,2.223983e-02
75%,2249.000000,8.627602e-01
max,2999.000000,1.728681e+00


In [20]:
# IQR 판단 기반 이상치 처리
# 원 데이터 카피
click_copy = click_data.copy()

In [22]:
# 1,3분위수 (Q1,Q3) 후하기
q1 = click_copy['num_click'].quantile(0.25)
q3 = click_copy['num_click'].quantile(0.75)

# IQR 구하기
iqr = (q3 - q1)
iqr

1512.25

In [23]:
#IQR 기반 이상치 제거하기
click_copy = click_copy[(click_copy['num_click'] > (q1 - 1.5 * iqr)) & (click_copy['num_click'] < (q3 + 1.5 * iqr))]
click_copy.describe()

,num_click
count,9993.000000
mean,1501.737416
std,866.173486
min,9.000000
25%,738.000000
50%,1521.000000
75%,2249.000000
max,2999.000000


In [24]:
# 이상치 제거후 데이터 확인
print(np.shape(click_copy))

(9993, 3)


## 이상치 대체하기

In [25]:
# 이상치 대체
# 원 데이터 copy
click_copy = click_data.copy()

In [26]:
# 대체할 기준 정의
max_click = 1000

In [28]:
# 정의 된 기준 으로 대체후 비교를 위해 컬럼 복사
click_copy['new_num_click'] = click_copy['num_click']

# 이상치 대체 (loc 함수에 주건부여하는 것 공부)
click_copy.loc[click_copy['new_num_click']> max_click, 'new_num_click'] = max_click
click_copy.describe()

,num_click,new_num_click
count,10000.000000,10000.000000
mean,1764.480600,832.669800
std,10150.132161,286.316357
min,9.000000,9.000000
25%,738.750000,738.750000
50%,1522.500000,1000.000000
75%,2251.000000,1000.000000
max,433992.000000,1000.000000


In [29]:
click_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   category       10000 non-null  object 
 1   Journal        10000 non-null  object 
 2   num_click      10000 non-null  float64
 3   new_num_click  10000 non-null  float64
dtypes: float64(2), object(2)
memory usage: 312.6+ KB
